In [31]:
import numpy as np
import pandas as pd
from pickle import load,dump
from nltk.tokenize import word_tokenize
# import gensim.downloader as api
# from gensim.matutils import softcossim
# from gensim import corpora

In [7]:
df = pd.read_csv('taxonomy.tsv', sep='\t')
df.head()

,Unique ID,Parent,Name,Tier 1,Tier 2,Tier 3,Tier 4,Unnamed: 7
0,150,150,Attractions,Attractions,NaN,NaN,NaN,NaN
1,151,150,Amusement and Theme Parks,Attractions,Amusement and Theme Parks,NaN,NaN,NaN
2,179,150,Bars & Restaurants,Attractions,Bars & Restaurants,NaN,NaN,NaN
3,181,150,Casinos & Gambling,Attractions,Casinos & Gambling,NaN,NaN,NaN
4,153,150,Historic Site and Landmark Tours,Attractions,Historic Site and Landmark Tours,NaN,NaN,NaN


In [10]:
tier0=df['Name']
tier1=df['Tier 1']
tier2=df['Tier 2']
tier3=df['Tier 3']
tier4=df['Tier 4']
tier5=df['Unnamed: 7']


In [16]:
print(f'Tier 0:\t{tier0.nunique()}')
print(f'Tier 1:\t{tier1.nunique()}')
print(f'Tier 2:\t{tier2.nunique()}')
print(f'Tier 3:\t{tier3.nunique()}')
print(f'Tier 4:\t{tier4.nunique()}')

Tier 0:	703
Tier 1:	40
Tier 2:	347
Tier 3:	256
Tier 4:	60


In [17]:
tier1.unique()

array(['Attractions', 'Automotive', 'Books and Literature',
       'Business and Finance', 'Careers', 'Communication', 'Crime',
       'Disasters', 'Education', 'Entertainment', 'Events',
       'Family and Relationships', 'Fine Art', 'Food & Drink', 'Genres',
       'Healthy Living', 'Hobbies & Interests', 'Holidays',
       'Home & Garden', 'Law', 'Maps & Navigation', 'Medical Health',
       'Music', 'Personal Celebrations & Life Events', 'Personal Finance',
       'Pets', 'Politics', 'Pop Culture', 'Productivity', 'Real Estate',
       'Religion & Spirituality', 'Science', 'Sensitive Topics',
       'Shopping', 'Sports', 'Style & Fashion', 'Technology & Computing',
       'Travel', 'Video Gaming', 'War and Conflicts'], dtype=object)

In [18]:
tier4.unique()

array([nan, 'Angel Investment', 'Bankruptcy', 'Business Loans',
       'Debt Factoring & Invoice Discounting', 'Mergers and Acquisitions',
       'Private Equity', 'Sale & Lease Back', 'Venture Capital',
       'Recalls', 'Professional School', 'Herbs and Supplements',
       'Holistic Health', 'Hormonal Disorders', 'Menopause',
       'Thyroid Disorders', 'First Aid', 'Birth Control', 'Infertility',
       'Pregnancy', 'Sexual Conditions', "Men's Jewelry and Watches",
       "Men's Business Wear", "Men's Casual Wear", "Men's Formal Wear",
       "Men's Outerwear", "Men's Sportswear",
       "Men's Underwear and Sleepwear", "Women's Glasses",
       "Women's Handbags and Wallets", "Women's Hats and Scarves",
       "Women's Jewelry and Watches", "Women's Business Wear",
       "Women's Casual Wear", "Women's Formal Wear",
       "Women's Intimates and Sleepwear", "Women's Outerwear",
       "Women's Sportswear", '3-D Graphics', 'Antivirus Software',
       'Browsers', 'Computer Animati

In [25]:
with open('model/word2index.pkl', 'rb') as pickle_file:
    word_index_Mapping = load(pickle_file)

word_index_Mapping['init'] = len(word_index_Mapping) + 1
index_word_Mapping = dict(map(reversed,word_index_Mapping.items()))
vocab_size = len(word_index_Mapping) + 1
print(vocab_size)
max_caption_length = 84

2966


In [52]:
%%time
#word embedding
glove_dir='glove/glove.6B.300d.txt'
embeddings_index = {}
embeddings_index_all = {}
f = open(glove_dir,encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index_all[word] = coefs

    if word in word_index_Mapping.keys():
        embeddings_index[word] = coefs

f.close()    
print(len(list(embeddings_index_all.keys())))


400000
CPU times: total: 17.3 s
Wall time: 30.1 s


In [27]:
%%time
print(set(embeddings_index.keys())^set(word_index_Mapping.keys()))

{'startSeq', '<pad>', 'endSeq'}
CPU times: total: 0 ns
Wall time: 0 ns


In [28]:
%%time

print(set(embeddings_index.keys())^set(tier0.unique()))

{'completing', 'canoe', 'athletic', 'perhaps', 'monkey', 'kayaking', 'Music Video', 'wheelchair', 'shoreline', 'Birth Control', 'consisting', 'photographer', 'silver', 'balancing', 'puck', 'age', 'dish', 'brown', 'watched', 'both', 'lighted', 'outfits', 'backpack', 'plants', 'outside', 'business', 'Classic Hits', 'microphones', 'drink', 'rollerblading', 'furniture', 'fluffy', 'Disasters', 'Auto Racing', 'lasso', 'cruise', 'lit', 'rug', 'Crime & Harmful Acts to Individuals, Society & Human Right Violations', 'team', 'works', 'softball', 'Environment', 'match', 'diver', 'toothbrush', 'eat', 'lake', 'enjoys', 'horns', 'roadway', 'coats', 'Gasoline Prices', 'games', 'appears', 'fur', 'Tennis', 'saw', 'ages', 'handstand', 'cuts', 'cones', 'computer', 'calm', 'possibly', 'surgical', 'dances', 'lunch', 'swimsuit', 'bathtub', 'Mental Health', 'waves', 'Drawing and Sketching', 'upset', 'Business and Finance', 'beach', 'crossing', 'state', 'university', 'pads', 'opposite', 'lot', 'pillar', 'hit'

In [61]:
t0 = [t.lower() for t in tier1.unique()] 
# open file
with open('tier1.txt', 'w+') as f:
     
    # write elements of list
    for items in t0:
        f.write('%s\n' %items)
     
    print("File written successfully")
 
 
# close the file
f.close()
covered = []
not_covered = []
for t in t0:
    tokens = word_tokenize(t)
    for tkn in tokens:
        if tkn in embeddings_index_all.keys():
            print(f'{tkn} found in glove')
            covered.append(tkn)
        else:
            print(f'{tkn} IS NOT in glove')
            not_covered.append(tkn)
print(f'Covered: {len(covered)}')
print(f'NOT Covered: {len(not_covered)}')

File written successfully
attractions found in glove
automotive found in glove
books found in glove
and found in glove
literature found in glove
business found in glove
and found in glove
finance found in glove
careers found in glove
communication found in glove
crime found in glove
disasters found in glove
education found in glove
entertainment found in glove
events found in glove
family found in glove
and found in glove
relationships found in glove
fine found in glove
art found in glove
food found in glove
& found in glove
drink found in glove
genres found in glove
healthy found in glove
living found in glove
hobbies found in glove
& found in glove
interests found in glove
holidays found in glove
home found in glove
& found in glove
garden found in glove
law found in glove
maps found in glove
& found in glove
navigation found in glove
medical found in glove
health found in glove
music found in glove
personal found in glove
celebrations found in glove
& found in glove
life found in gl

In [55]:
def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(embeddings_index_all[word1], embeddings_index_all[word2]))

In [63]:
cosine_distance_between_two_words('man','spider')

0.27324068546295166

In [70]:
from scenedetect import detect, AdaptiveDetector, split_video_ffmpeg
scene_list = detect('sample.mp4', AdaptiveDetector())
split_video_ffmpeg('sample.mp4', scene_list)



ffmpeg could not be found on the system. Please install ffmpeg to enable video output support.


0